In [70]:
import pandas as pd

# Mapowania

In [71]:
mapowanie_pkd_2007_2025 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="MAP_PKD_2007_2025")
# mapowanie_pkd_2025_2007 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="MAP_PKD_2025_2007")
pkd_2025 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="PKD_2025")
# pkd_2007 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="PKD_2007")

In [72]:
mapowanie_pkd_2007_2025_with_og = pd.concat([
    mapowanie_pkd_2007_2025,
    pd.DataFrame({'symbol_2007': ['OG'], 'symbol_2025': ['OG']})
], ignore_index=True)

In [73]:
pkd_2025

,typ,symbol,nazwa
0,SEKCJA,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO"
1,DZIAŁ,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT..."
2,GRUPA,01.1,Uprawy rolne inne niż wieloletnie
3,KLASA,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
4,PODKLASA,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
...,...,...,...
1770,SEKCJA,V,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1771,DZIAŁ,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1772,GRUPA,99.0,Organizacje i zespoły eksterytorialne
1773,KLASA,99.00,Organizacje i zespoły eksterytorialne


In [74]:
pkd_2025_with_og = pd.concat([
    pkd_2025,
    pd.DataFrame({'typ': ['OGÓŁEM'], 'symbol': ['OG'], 'nazwa': ['OGÓŁEM']})
], ignore_index=True)
pkd_2025_with_og

,typ,symbol,nazwa
0,SEKCJA,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO"
1,DZIAŁ,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT..."
2,GRUPA,01.1,Uprawy rolne inne niż wieloletnie
3,KLASA,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
4,PODKLASA,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
...,...,...,...
1771,DZIAŁ,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1772,GRUPA,99.0,Organizacje i zespoły eksterytorialne
1773,KLASA,99.00,Organizacje i zespoły eksterytorialne
1774,PODKLASA,99.00.Z,Organizacje i zespoły eksterytorialne


# Upadlosci - KRZ_PKD
krz_pkd.csv - przypisanie liczby upadłości firm z Krajowego Rejestru Zadłużonych do PKD
(2007) od 2018 roku -
**poziom podklasy**

In [75]:
upadlosci = pd.read_csv(r"..\data\krz_pkd.csv", sep=";", encoding="utf-8")

In [76]:
upadlosci["pkd_formatted"] = (
    upadlosci["pkd"]
    .str.strip()
    .str.upper()
    .str.replace(r"^(\d{2})(\d{2})([A-Z])$", r"\1.\2.\3", regex=True)
)

In [77]:
upadlosci["pkd_formatted_no_letter"] = upadlosci["pkd_formatted"].str.replace(r"\.[A-Z]$", "", regex=True)
upadlosci["pkd_letter"] = upadlosci["pkd_formatted"].str.extract(r"\.([A-Z])$")

upadlosci_mapped = upadlosci.merge(
    mapowanie_pkd_2007_2025[["symbol_2007", "symbol_2025"]],
    how="left",
    left_on="pkd_formatted_no_letter",
    right_on="symbol_2007"
)
unmapped_mask = upadlosci_mapped["symbol_2025"].isna()
if unmapped_mask.any():
    missing_codes = (
        upadlosci_mapped.loc[unmapped_mask, "pkd_formatted_no_letter"]
        .dropna()
        .unique()
        .tolist()
    )
    print(f"Warning: Niezmapowane kody PKD 2007: {missing_codes}")
    upadlosci_mapped = upadlosci_mapped[~unmapped_mask]


In [78]:
upadlosci_mapped["pkd_2025"] = upadlosci_mapped["symbol_2025"] + '.' + upadlosci_mapped["pkd_letter"]

In [79]:
upadlosci_final = upadlosci_mapped[["rok", "pkd_2025", "liczba_upadlosci"]]

In [80]:
upadlosci_final

,rok,pkd_2025,liczba_upadlosci
0,2018,01.11.Z,6
1,2018,01.13.Z,4
2,2018,01.19.Z,2
3,2018,01.24.Z,1
4,2018,01.29.Z,1
...,...,...,...
2540,2024,96.21.Z,12
2541,2024,96.22.Z,12
2542,2024,96.30.Z,1
2543,2024,93.13.Z,1


# Wskaznik finansowy
wszystkie poziomy oprócz podklas

In [81]:
wskaznik_finansowy = pd.read_csv(r"..\data\wsk_fin.csv", sep=";", encoding="utf-8")

In [82]:
#  OG - ogółem; SEK_A - sekctor; 
# wskaznik_finansowy

In [83]:
wskaznik_finansowy["PKD_formatted"] = wskaznik_finansowy["PKD"].str.replace(r"^SEK_", "", regex=True).str.rstrip('.')

In [84]:
wskaznik_finansowy = wskaznik_finansowy.drop(columns=["NAZWA_PKD", "NUMER_NAZWA_PKD","PKD"])

In [85]:
wskaznik_finansowy_mapped = wskaznik_finansowy.join(
    mapowanie_pkd_2007_2025_with_og[["symbol_2007", "symbol_2025"]].set_index("symbol_2007"),
    on="PKD_formatted"
)
unmapped_mask = wskaznik_finansowy_mapped["symbol_2025"].isna()
if unmapped_mask.any():
    missing_codes = (
        wskaznik_finansowy_mapped.loc[unmapped_mask, "PKD_formatted"]
        .dropna()
        .unique()
        .tolist()
    )
    raise ValueError(f"Niezmapowane kody PKD 2007: {missing_codes}")

In [86]:
wskaznik_finansowy_mapped = wskaznik_finansowy_mapped.drop(columns=["PKD_formatted"])

In [87]:
wskaznik_finansowy_transposed = wskaznik_finansowy_mapped.melt(
    id_vars=['symbol_2025', 'WSKAZNIK'],
    var_name='rok',
    value_name='wartosc'
)
wskaznik_finansowy_transposed['rok'] = wskaznik_finansowy_transposed['rok'].astype(int)

In [88]:
wskaznik_finansowy_transposed['wartosc'] = wskaznik_finansowy_transposed['wartosc'].replace('bd', pd.NA)

In [89]:
wskaznik_finansowy_final = wskaznik_finansowy_transposed.rename(columns={'symbol_2025': 'pkd_2025'})

In [90]:
wskaznik_finansowy_final


,pkd_2025,WSKAZNIK,rok,wartosc
0,OG,EN Liczba jednostek gospodarczych,2005,"46 396,00"
1,OG,PEN Liczba rentownych jednostek gospodarczych,2005,"35 860,00"
2,OG,GS Przychody ogółem,2005,"1 679 774,30"
3,OG,PNPM Przychody netto,2005,<NA>
4,OG,GS (I) Przychody netto ze sprzedaży i zrównane...,2005,"1 614 314,24"
...,...,...,...,...
418555,99.0,STL Zobowiązania krótkoterminowe,2024,<NA>
418556,99.0,LTC Długoterminowe kredyty bankowe,2024,<NA>
418557,99.0,STC Krótkoterminowe kredyty bankowe,2024,<NA>
418558,99.0,INV Zapasy,2024,<NA>


In [91]:
upadlosci_final['WSKAZNIK'] = 'Upadłość'
upadlosci_final = upadlosci_final.rename(columns={'liczba_upadlosci': 'wartosc'})

C:\Users\tusiu\AppData\Local\Temp\ipykernel_2196\2794861416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upadlosci_final['WSKAZNIK'] = 'Upadłość'


In [92]:
upadlosci_final

,rok,pkd_2025,wartosc,WSKAZNIK
0,2018,01.11.Z,6,Upadłość
1,2018,01.13.Z,4,Upadłość
2,2018,01.19.Z,2,Upadłość
3,2018,01.24.Z,1,Upadłość
4,2018,01.29.Z,1,Upadłość
...,...,...,...,...
2540,2024,96.21.Z,12,Upadłość
2541,2024,96.22.Z,12,Upadłość
2542,2024,96.30.Z,1,Upadłość
2543,2024,93.13.Z,1,Upadłość


In [93]:
combined_data = pd.concat([wskaznik_finansowy_final, upadlosci_final], ignore_index=True)
combined_data

,pkd_2025,WSKAZNIK,rok,wartosc
0,OG,EN Liczba jednostek gospodarczych,2005,"46 396,00"
1,OG,PEN Liczba rentownych jednostek gospodarczych,2005,"35 860,00"
2,OG,GS Przychody ogółem,2005,"1 679 774,30"
3,OG,PNPM Przychody netto,2005,<NA>
4,OG,GS (I) Przychody netto ze sprzedaży i zrównane...,2005,"1 614 314,24"
...,...,...,...,...
421093,96.21.Z,Upadłość,2024,12
421094,96.22.Z,Upadłość,2024,12
421095,96.30.Z,Upadłość,2024,1
421096,93.13.Z,Upadłość,2024,1


In [94]:
# Derive dictionary for WSKAZNIK values and attach their indices
unique_wskaznik = (
    combined_data["WSKAZNIK"].dropna().sort_values().unique()
)
wskaznik_dict = {value: idx for idx, value in enumerate(unique_wskaznik)}
wskaznik_dictionary_table = (
    pd.DataFrame(
        [(idx, value) for value, idx in wskaznik_dict.items()],
        columns=["WSKAZNIK_INDEX", "WSKAZNIK"]
    )
    .sort_values("WSKAZNIK_INDEX")
    .reset_index(drop=True)
)
combined_data["WSKAZNIK_INDEX"] = combined_data["WSKAZNIK"].map(wskaznik_dict)
combined_data.drop(columns=["WSKAZNIK"], inplace=True)
wskaznik_dictionary_table

,WSKAZNIK_INDEX,WSKAZNIK
0,0,C Środki pieniężne i pap. wart.
1,1,CF Nadwyżka finansowa
2,2,DEPR Amortyzacja
3,3,EN Liczba jednostek gospodarczych
4,4,GS (I) Przychody netto ze sprzedaży i zrównane...
5,5,GS Przychody ogółem
6,6,INV Zapasy
7,7,IO Wartość nakładów inwestycyjnych
8,8,IP Odsetki do zapłacenia
9,9,LTC Długoterminowe kredyty bankowe


In [95]:
combined_data

,pkd_2025,rok,wartosc,WSKAZNIK_INDEX
0,OG,2005,"46 396,00",3
1,OG,2005,"35 860,00",15
2,OG,2005,"1 679 774,30",5
3,OG,2005,<NA>,16
4,OG,2005,"1 614 314,24",4
...,...,...,...,...
421093,96.21.Z,2024,12,24
421094,96.22.Z,2024,12,24
421095,96.30.Z,2024,1,24
421096,93.13.Z,2024,1,24


In [96]:
pkd_2025_with_og['typ']

0         SEKCJA
1          DZIAŁ
2          GRUPA
3          KLASA
4       PODKLASA
          ...   
1771       DZIAŁ
1772       GRUPA
1773       KLASA
1774    PODKLASA
1775      OGÓŁEM
Name: typ, Length: 1776, dtype: object

In [97]:
# Build dictionary for PKD typ values and map indices
unique_typ_values = pkd_2025_with_og["typ"].dropna().unique()
typ_dict = {value: idx for idx, value in enumerate(unique_typ_values)}
pkd_typ_dictionary_table = (
    pd.DataFrame(
        [(idx, value) for value, idx in typ_dict.items()],
        columns=["TYP_INDEX", "typ"]
    )
    .sort_values("TYP_INDEX")
    .reset_index(drop=True)
)
pkd_2025_with_og["TYP_INDEX"] = pkd_2025_with_og["typ"].map(typ_dict)
pkd_2025_with_og.drop(columns=["typ"], inplace=True)
pkd_typ_dictionary_table

,TYP_INDEX,typ
0,0,SEKCJA
1,1,DZIAŁ
2,2,GRUPA
3,3,KLASA
4,4,PODKLASA
5,5,OGÓŁEM


In [98]:
pkd_2025_with_og

,symbol,nazwa,TYP_INDEX
0,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO",0
1,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT...",1
2,01.1,Uprawy rolne inne niż wieloletnie,2
3,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",3
4,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",4
...,...,...,...
1771,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE,1
1772,99.0,Organizacje i zespoły eksterytorialne,2
1773,99.00,Organizacje i zespoły eksterytorialne,3
1774,99.00.Z,Organizacje i zespoły eksterytorialne,4


In [99]:
# Normalize PKD codes for consistent dictionary joins
def normalize_pkd_code(value):
    if pd.isna(value):
        return pd.NA
    code = str(value).strip().upper()
    if code.endswith('.0'):
        code = code[:-2]
    return code
pkd_2025_with_og["symbol_normalized"] = pkd_2025_with_og["symbol"].map(normalize_pkd_code)
combined_data["pkd_2025_normalized"] = combined_data["pkd_2025"].map(normalize_pkd_code)

In [100]:
# Add PKD identifiers and map them into the fact table
unique_pkd_values = pkd_2025_with_og["symbol_normalized"].dropna().sort_values().unique()
pkd_dict = {value: idx for idx, value in enumerate(unique_pkd_values)}
pkd_2025_with_og["PKD_INDEX"] = pkd_2025_with_og["symbol_normalized"].map(pkd_dict)
pkd_dictionary_table = (
    pkd_2025_with_og[["PKD_INDEX", "symbol", "nazwa", "TYP_INDEX"]]
    .drop_duplicates(subset=["PKD_INDEX"])
    .sort_values("PKD_INDEX")
    .reset_index(drop=True)
)
pkd_dictionary_table

,PKD_INDEX,symbol,nazwa,TYP_INDEX
0,0,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT...",1
1,1,01.1,Uprawy rolne inne niż wieloletnie,2
2,2,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",3
3,3,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",4
4,4,01.12,Uprawa ryżu,3
...,...,...,...,...
1759,1759,R,OPIEKA ZDROWOTNA I POMOC SPOŁECZNA,0
1760,1760,S,"DZIAŁALNOŚĆ ZWIĄZANA Z KULTURĄ, SPORTEM I REKR...",0
1761,1761,T,POZOSTAŁA DZIAŁALNOŚĆ USŁUGOWA,0
1762,1762,U,GOSPODARSTWA DOMOWE ZATRUDNIAJĄCE PRACOWNIKÓW ...,0


In [101]:
combined_data["PKD_INDEX"] = combined_data["pkd_2025_normalized"].map(pkd_dict)
combined_data

,pkd_2025,rok,wartosc,WSKAZNIK_INDEX,pkd_2025_normalized,PKD_INDEX
0,OG,2005,"46 396,00",3,OG,1756.0
1,OG,2005,"35 860,00",15,OG,1756.0
2,OG,2005,"1 679 774,30",5,OG,1756.0
3,OG,2005,<NA>,16,OG,1756.0
4,OG,2005,"1 614 314,24",4,OG,1756.0
...,...,...,...,...,...,...
421093,96.21.Z,2024,12,24,96.21.Z,1712.0
421094,96.22.Z,2024,12,24,96.22.Z,1714.0
421095,96.30.Z,2024,1,24,96.30.Z,1719.0
421096,93.13.Z,2024,1,24,93.13.Z,1654.0


In [102]:
missing_pkd_mask = combined_data["PKD_INDEX"].isna()
if missing_pkd_mask.any():
    missing_pkd_codes = (
        combined_data.loc[missing_pkd_mask, "pkd_2025_normalized"]
        .dropna()
        .unique()
        .tolist()
    )
    # raise ValueError(f"Niezmapowane kody PKD: {missing_pkd_codes}")
    print(f"Warning: Niezmapowane kody PKD: {missing_pkd_codes}")
    combined_data = combined_data[~missing_pkd_mask]

In [103]:
columns_to_drop = [col for col in ["pkd_2025", "pkd_2025_normalized"] if col in combined_data.columns]
if columns_to_drop:
    combined_data.drop(columns=columns_to_drop, inplace=True)
if "symbol_normalized" in pkd_2025_with_og.columns:
    pkd_2025_with_og.drop(columns=["symbol_normalized"], inplace=True)

C:\Users\tusiu\AppData\Local\Temp\ipykernel_2196\95792252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data.drop(columns=columns_to_drop, inplace=True)


In [104]:
import os

# Save final tables to CSV files
# Create results directory if it doesn't exist
os.makedirs(r"..\results", exist_ok=True)

wskaznik_dictionary_table.to_csv(r"..\results\wskaznik_dictionary.csv", sep=";", encoding="utf-8", index=False)
pkd_dictionary_table.to_csv(r"..\results\pkd_dictionary.csv", sep=";", encoding="utf-8", index=False)
pkd_typ_dictionary_table.to_csv(r"..\results\pkd_typ_dictionary.csv", sep=";", encoding="utf-8", index=False)
combined_data.to_csv(r"..\results\kpi-value-table.csv", sep=";", encoding="utf-8", index=False)

print("All tables saved successfully!")

All tables saved successfully!


# final tables
* wskaznik_dictionary_table - dim
* combined_data - fact
* pkd_dictionary_table - dim pkd
* pkd_typ_dictionary_table - dim hierarchy